## DTU



In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import netCDF4 as nc
import xarray as xr
import os
import math
from matplotlib import pyplot as plt
import xesmf as xe
import pandas as pd
import pickle
from nch import * # saved generalized TCH function in python script for readability
from itertools import combinations
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import shutil

In [3]:
inputdir='/glade/work/clittle/p2521/input/'
outputdir='/glade/work/clittle/p2521/output/'
obsdir='obs/'
modeldir='model/'

## Regrid model and data

In [27]:
# Bathymetry data
bathy = np.load(inputdir+obsdir+'GEBCO_1_12.npy')

bathy_lat = np.arange(-90,90+1/12,1/12)
bathy_lon = np.arange(-180,180,1/12)

ds_bathy = xr.Dataset(data_vars=dict(depth=(["latitude", "longitude"], bathy)), 
                      coords=dict(latitude=("latitude", bathy_lat), longitude=("longitude", bathy_lon)),
                      attrs=dict(description="GEBCO bathymetry (depth below geoid) data")                      
)
ds_bathy = ds_bathy.assign_coords(longitude=((360 + (ds_bathy.longitude % 360)) % 360))
ds_bathy = ds_bathy.roll(longitude=int(len(ds_bathy['longitude']) / 2),roll_coords=True)
ds_bathy.to_netcdf(inputdir+obsdir+"GEBCO.nc")

In [6]:
%%time
# Reformat DTU 

dtu = np.genfromtxt(inputdir+obsdir+"dtuuh22mdt.xyz", dtype=None, skip_header=1) # column: lon,lat,dtu,err(?); the gridded data online cause errors when reading in
# mask according to column 4, where values are 9.999
dtu[dtu[:,3]==9.999, 2] = np.nan
dtu[dtu[:,3]==9.999, 3] = np.nan
# write dtu into gridded data
lat_dtu = np.arange(85,-79-0.125,-0.125)
lon_dtu = np.arange(0,360+0.125,0.125)
dtuuh = np.zeros((len(lat_dtu),len(lon_dtu)))*np.nan
dtuuherr = np.zeros((len(lat_dtu),len(lon_dtu)))*np.nan
for i in (np.arange(len(lat_dtu))):
    idx0 = np.where(dtu[:,1]==lat_dtu[i])[0] # indices in dtu with latitudes i
    j = dtu[idx0,0] # longitudes with latitude i
    idx = np.where(np.isin(lon_dtu,j))[0]   # indices in dtuuh corresponding to longitudes j
    dtuuh[i,idx] = dtu[idx0,2]  
    dtuuherr[i,idx] = dtu[idx0,3]
    # print(i)
del i,j,idx0,idx
dtuuh = np.flipud(dtuuh)
dtuuherr = np.flipud(dtuuherr)
lat_dtu = np.arange(-79,85+0.125,0.125)

# write dtu into xarray dataset
ds_obs_dtu = xr.Dataset(
    data_vars=dict(
        mdt=(["latitude", "longitude"], dtuuh),
        err_mdt=(["latitude", "longitude"], dtuuherr),
        ),
    coords=dict(
        longitude=(["longitude"], lon_dtu),
        latitude=(["latitude"], lat_dtu),
        ),
    )

del dtu, lat_dtu, lon_dtu, dtuuh, dtuuherr


CPU times: user 26.7 s, sys: 3.4 s, total: 30.1 s
Wall time: 43.6 s


In [25]:
ds_obs_dtu.to_netcdf(inputdir+obsdir+"dtuuh22mdt.nc")

In [28]:
# ds_obs_cnes = xr.open_dataset(inputdir+obsdir+"cnescls22mdt.nc")
# ds_obs_cnes=ds_obs_cnes.drop_vars('time')
# ds_obs_cnes = ds_obs_cnes.assign_coords(longitude=((360 + (ds_obs_cnes.longitude % 360)) % 360))
# ds_obs_cnes = ds_obs_cnes.roll(longitude=int(len(ds_obs_cnes['longitude']) / 2),roll_coords=True)
# ds_obs_cnes_out=ds_obs_cnes.isel(time=0)
# ds_obs_cnes_out

In [9]:
# ds_obs_dtu = xr.open_dataset(inputdir+obsdir+"dtuuh22mdt.nc")
# ds_obs_dtu

<xarray.Dataset>
Dimensions:    (latitude: 1313, longitude: 2881)
Coordinates:
  * longitude  (longitude) float64 0.0 0.125 0.25 0.375 ... 359.8 359.9 360.0
  * latitude   (latitude) float64 -79.0 -78.88 -78.75 ... 84.75 84.88 85.0
Data variables:
    mdt        (latitude, longitude) float64 ...
    err_mdt    (latitude, longitude) float64 ...